In [1]:
import pandas as pd
import numpy as np

In [2]:
ids_df = pd.read_csv("./IDS_mapping.csv")
df = pd.read_csv("./diabetic_data.csv")

In [3]:
#splitting data of ids file
admission_type_id_df = ids_df.iloc[:8]

discharge_disposition_id_df = ids_df.iloc[10:40]
discharge_disposition_id_df = discharge_disposition_id_df.rename(columns={'admission_type_id': 'discharge_disposition_id'})


admission_source_id_df = ids_df.iloc[42:]
admission_source_id_df = admission_source_id_df.rename(columns={'admission_type_id': 'admission_source_id'})


In [4]:
discharge_disposition_id_df

,discharge_disposition_id,description
10,1,Discharged to home
11,2,Discharged/transferred to another short term h...
12,3,Discharged/transferred to SNF
13,4,Discharged/transferred to ICF
14,5,Discharged/transferred to another type of inpa...
15,6,Discharged/transferred to home with home healt...
16,7,Left AMA
17,8,Discharged/transferred to home under care of H...
18,9,Admitted as an inpatient to this hospital
19,10,Neonate discharged to another hospital for neo...


In [5]:
df.isnull().sum()

encounter_id                    0
patient_nbr                     0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

### we have,

1. 30248 repeated patients
2. all unique encounter_id
3. 2273 patients with unknown race marked by '?'
4. 3 records with 'Unknown/Invalid' gender
5. all records of age
6. 98569 unknown weights marked by '?'
7. 9 admission_type_id
8. discharge_disposition_id {21, 26} missing
9. admission_source_id {12, 15, 16, 18, 19, 21, 23, 24} missing
10. Integer: number of days between admission and discharge
11. 40256 unknown payer_code marked by '?' with 17 distinct providers #why dataset info has 23 distinct values??
12. 49949 unknown medical_specialty marked by '?' with 72 distinct specialist  #why dataset info has 84 distinct values??
13. Number of lab tests performed during the encounter with min 1 and max 132
14. Number of procedures (other than lab tests) performed during the encounter min 0 and max 6
15. Number of distinct generic names administered during the encounter min 1 and max 81
16. Number of outpatient visits of the patient in the year preceding the encounter(vistis without hospitalizing).
17. Number of emergency visits of the patient in the year preceding the encounter
18. The number of times a patient is admitted to a hospital for medical care
19. The primary diagnosis (coded as first three digits of ICD9); 717 distinct values #why dataset info has 848 distinct values??
20. Secondary diagnosis (coded as first three digits of ICD9); 749 distinct values #why dataset info has 923 distinct values??
21. Additional secondary diagnosis (coded as first three digits of ICD9); 790 distinct values #why dataset info has 954 distinct values??
22. Number of diagnoses entered to the system
23. Indicates the range of the result or if the test was not taken. Values: >200, >300, normal, and none if not measured
24. If the test was not taken. Values: >8 if the result was greater than 8%, >7 if the result was greater than 7% but less than 8%, normal if the result was less than 7%, and none if not measured.
25. Whether the drug was prescribed or there was a change in the dosage.
26. Whether the drug was prescribed or there was a change in the dosage.
27. Whether the drug was prescribed or there was a change in the dosage.
28. Whether the drug was prescribed or there was a change in the dosage.
29. Whether the drug was prescribed or there was a change in the dosage.
30. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
31. Whether the drug was prescribed or there was a change in the dosage.
32. Whether the drug was prescribed or there was a change in the dosage.
33. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
34. Whether the drug was prescribed or there was a change in the dosage.
35. Whether the drug was prescribed or there was a change in the dosage.
36. Whether the drug was prescribed or there was a change in the dosage.
37. Whether the drug was prescribed or there was a change in the dosage.
38. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
39. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 3 values
40. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 1 value i.e No
41. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 1 value i.e No
42. Whether the drug was prescribed or there was a change in the dosage.
43. Whether the drug was prescribed or there was a change in the dosage.
44. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
45. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
46. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
47. Whether the drug was prescribed or there was a change in the dosage. # why does dataset have 2 values
48. Indicates if there was a change in diabetic medications.
49. Indicates if there was any diabetic medication prescribed.
50. Days to inpatient readmission. {'<30', '>30', 'No'}

In [6]:
# dropping 'max_glu_serum', 'A1Cresult' columns as the null values are too high
df = df.drop(['max_glu_serum', 'A1Cresult'], axis = 1)

In [7]:
# '?' existing in the data
df.isin(['?']).sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [8]:
# print(df[df['medical_specialty'] == '?'].shape)

# # has '?' in 2273 columns

In [9]:
# dropping 'weight', 'payer_code' and 'medical_specialty' columns - a lot of '?' values exist
df = df.drop(['weight', 'payer_code', 'medical_specialty'], axis = 1)

In [10]:
df.replace('?', np.nan, inplace = True)

In [11]:
df.isin(['?']).sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide

In [12]:
df.isnull().sum()

encounter_id                   0
patient_nbr                    0
race                        2273
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
diag_1                        21
diag_2                       358
diag_3                      1423
number_diagnoses               0
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitazone                  0
acarbose  

In [13]:
# As we have very less rows with null values - we can drop those rows
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide

In [15]:
df.shape

(98053, 45)

In [16]:
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone        

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98053 entries, 1 to 101765
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              98053 non-null  int64 
 1   patient_nbr               98053 non-null  int64 
 2   race                      98053 non-null  object
 3   gender                    98053 non-null  object
 4   age                       98053 non-null  object
 5   admission_type_id         98053 non-null  int64 
 6   discharge_disposition_id  98053 non-null  int64 
 7   admission_source_id       98053 non-null  int64 
 8   time_in_hospital          98053 non-null  int64 
 9   num_lab_procedures        98053 non-null  int64 
 10  num_procedures            98053 non-null  int64 
 11  num_medications           98053 non-null  int64 
 12  number_outpatient         98053 non-null  int64 
 13  number_emergency          98053 non-null  int64 
 14  number_inpatient          

In [18]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [19]:
df['gender'].value_counts()

gender
Female             52833
Male               45219
Unknown/Invalid        1
Name: count, dtype: int64

In [20]:
df['metformin'].value_counts()

metformin
No        78808
Steady    17677
Up         1017
Down        551
Name: count, dtype: int64

In [21]:
# columns for which we need value_counts
columns_of_interest = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed'] 

# Apply value_counts to the these columns
value_counts_dict = {col: df[col].value_counts() for col in columns_of_interest}

# Print the results
for col, counts in value_counts_dict.items():
    print(f"Value counts for column '{col}':")
    print(counts)
    print()


Value counts for column 'metformin':
metformin
No        78808
Steady    17677
Up         1017
Down        551
Name: count, dtype: int64

Value counts for column 'repaglinide':
repaglinide
No        96530
Steady     1371
Up          107
Down         45
Name: count, dtype: int64

Value counts for column 'nateglinide':
nateglinide
No        97362
Steady      657
Up           23
Down         11
Name: count, dtype: int64

Value counts for column 'chlorpropamide':
chlorpropamide
No        97970
Steady       76
Up            6
Down          1
Name: count, dtype: int64

Value counts for column 'glimepiride':
glimepiride
No        93066
Steady     4488
Up          315
Down        184
Name: count, dtype: int64

Value counts for column 'acetohexamide':
acetohexamide
No        98052
Steady        1
Name: count, dtype: int64

Value counts for column 'glipizide':
glipizide
No        85769
Steady    10991
Up          752
Down        541
Name: count, dtype: int64

Value counts for column 'glyburide':

In [22]:
# List of categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Threshold for low variance
threshold = 0.999999999

# Identify low-variance columns
low_variance_cols = [
    col for col in categorical_columns
    if df[col].value_counts(normalize=True).max() > threshold
]

# Print low-variance columns
print(f"Low-variance columns: {low_variance_cols}")

Low-variance columns: ['examide', 'citoglipton', 'metformin-rosiglitazone']


In [23]:
# dropping these columns which have very very low variance 
df = df.drop(['examide', 'citoglipton', 'metformin-rosiglitazone'], axis =1)

In [24]:
# Summing all the visits together
df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']

In [25]:
keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
for col in keys:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)

In [26]:
df['numchange'] = 0
for col in keys:
    colname = str(col) + 'temp'
    df['numchange'] = df['numchange'] + df[colname]
    del df[colname]

df['numchange'].value_counts()

numchange
0    71310
1    25350
2     1281
3      107
4        5
Name: count, dtype: int64

Few category types can be replaced by the other as they almost mean the same

In [27]:
df['admission_type_id'] = df['admission_type_id'].replace(2,1)
df['admission_type_id'] = df['admission_type_id'].replace(7,1)
df['admission_type_id'] = df['admission_type_id'].replace(6,5)
df['admission_type_id'] = df['admission_type_id'].replace(8,5)

In [28]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(6,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(8,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(9,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(13,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(3,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(4,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(5,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(14,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(22,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(23,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(24,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(12,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(15,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(16,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(17,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(25,18)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(26,18)

In [29]:
df['admission_source_id'] = df['admission_source_id'].replace(2,1)
df['admission_source_id'] = df['admission_source_id'].replace(3,1)
df['admission_source_id'] = df['admission_source_id'].replace(5,4)
df['admission_source_id'] = df['admission_source_id'].replace(6,4)
df['admission_source_id'] = df['admission_source_id'].replace(10,4)
df['admission_source_id'] = df['admission_source_id'].replace(22,4)
df['admission_source_id'] = df['admission_source_id'].replace(25,4)
df['admission_source_id'] = df['admission_source_id'].replace(15,9)
df['admission_source_id'] = df['admission_source_id'].replace(17,9)
df['admission_source_id'] = df['admission_source_id'].replace(20,9)
df['admission_source_id'] = df['admission_source_id'].replace(21,9)
df['admission_source_id'] = df['admission_source_id'].replace(13,11)
df['admission_source_id'] = df['admission_source_id'].replace(14,11)

In [30]:
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)
df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)
# keys is the same as before
for col in keys:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)

/var/folders/sl/p2bldhbn72jg5x6t45zg0_wm0000gn/T/ipykernel_16114/4019121076.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['change'] = df['change'].replace('No', 0)
/var/folders/sl/p2bldhbn72jg5x6t45zg0_wm0000gn/T/ipykernel_16114/4019121076.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['diabetesMed'] = df['diabetesMed'].replace('No', 0)
/var/folders/sl/p2bldhbn72jg5x6t45zg0_wm0000gn/T/ipykernel_16114/4019121076.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ve

In [31]:
for i in range(0,10):
    df['age'] = df['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')', i+1)
df['age'].value_counts()

/var/folders/sl/p2bldhbn72jg5x6t45zg0_wm0000gn/T/ipykernel_16114/3167922647.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['age'] = df['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')', i+1)


age
8     25306
7     21809
9     16702
6     16697
5      9265
4      3548
10     2717
3      1478
2       466
1        65
Name: count, dtype: int64

In [32]:
df.shape

(98053, 44)